### Check influence of different parameters in performance. 

Check influence of:
- specifying different chunk sizes and chunking along different dimensions
- use already standardized data --> does it save memory?
- use ``` .persist()``` to load data in a distributed way and speed up reading



In [1]:
chunk_size = 483

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import sys
sys.path.append('/'.join(sys.path[0].split('/')[:-1]))

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import healpy as hp
import random

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

from modules.utils import train_model_2steps, init_device
from modules.data import WeatherBenchDatasetXarrayHealpix
from modules.healpix_models import UNetSphericalHealpix
from modules.test import create_iterative_predictions_healpix
from modules.test import compute_rmse_healpix
from modules.plotting import plot_rmses

datadir = "../data/healpix/"
input_dir = datadir + "5.625deg_nearest/"
model_save_path = datadir + "models/"
pred_save_path = datadir + "predictions/"

train_years = ('1979', '2012')
val_years = ('2013', '2016')
test_years = ('2017', '2018')

nodes = 12*16*16
max_lead_time = 5*24
lead_time = 6
out_features = 2
nb_timesteps = 2
len_sqce = 2
# define time resolution
delta_t = 6

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2,4"
gpu = [0,1]
num_workers = 10
pin_memory = True
batch_size = 95

nb_epochs = 10
learning_rate = 8e-3

obs = xr.open_mfdataset(pred_save_path + 'observations_nearest.nc', combine='by_coords', chunks={'time':483})
#rmses_weyn = xr.open_dataset(datadir + 'metrics/rmses_weyn.nc')

In [16]:
from modules.data import WeatherBenchDatasetIterative
class WeatherBenchDatasetXarrayHealpixTemp(Dataset):
    
    """ Dataset used for graph models (1D), where data is loaded from stored numpy arrays.
    
    Parameters
    ----------
    ds : xarray Dataset
        Dataset containing the input data
    out_features : int
        Number of output features
    delta_t : int
        Temporal spacing between samples in temporal sequence (in hours)
    len_sqce : int
        Length of the input and output (predicted) sequences
    years : tuple(str)
        Years used to split the data
    nodes : float
        Number of nodes each sample has
    max_lead_time : int
        Maximum lead time (in case of iterative predictions) in hours
    load : bool
        If true, load dataset to RAM
    mean : np.ndarray of shape 2
        Mean to use for data normalization. If None, mean is computed from data
    std : np.ndarray of shape 2
        std to use for data normalization. If None, mean is computed from data
    """
        
    def __init__(self, ds, out_features, delta_t, len_sqce, years, nodes, nb_timesteps, 
                 max_lead_time=None, load=True, mean=None, std=None):
        
        
        self.delta_t = delta_t
        self.len_sqce = len_sqce
        self.years = years
        
        self.nodes = nodes
        self.out_features = out_features
        self.max_lead_time = max_lead_time
        self.nb_timesteps = nb_timesteps
        
        self.data = ds.to_array(dim='level', name='Dataset').transpose('time', 'node', 'level')
        self.in_features = self.data.shape[-1]
        
        self.mean = self.data.mean(('time', 'node')).compute() if mean is None else mean
        self.std = self.data.std(('time', 'node')).compute() if std is None else std
        
        
        
        # Count total number of samples
        total_samples = self.data.shape[0]        
        
        if max_lead_time is None:
            self.n_samples = total_samples - (len_sqce+1) * delta_t
        else:
            self.n_samples = total_samples - (len_sqce+1) * delta_t - max_lead_time
        
        # Normalize
        self.data = (self.data - self.mean.to_array(dim='level')) / self.std.to_array(dim='level')
        
        # Create indexes
        #self.idxs = [[[[sample_idx + delta_t*k for k in range(len_sqce)], sample_idx + delta_t * len_sqce], 
        #              [sample_idx + delta_t * len_sqce, sample_idx + delta_t * (len_sqce+1)]] 
        #             for sample_idx in range(self.n_samples)]
        
        self.idxs = np.array(range(self.n_samples))
        
        
        #if load: 
        #    print('Loading data into RAM')
        #    self.data.load()
            
        
    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, idx):
        """ Returns sample and label corresponding to an index as torch.Tensor objects
            The return tensor shapes are (for the sample and the label): [n_vertex, len_sqce, n_features]
        """
        idx_data = self.idxs[idx]
        #1,0,2
        
        #batch[0] --> (batch_size, num_nodes, n_features*len_sq)
        X = (
            torch.tensor([self.data.isel(time=[idx_d + self.delta_t*k for k in range(self.len_sqce)]).values for idx_d in idx_data], \
                         dtype=torch.float).permute(0, 2,1,3).reshape(len(idx), self.nodes, -1),\
            
             torch.tensor([self.data.isel(time=[idx_d + self.delta_t * self.len_sqce]).values[:,:,self.out_features:] for idx_d in idx_data],\
                         dtype=torch.float).reshape(len(idx), self.nodes, -1)
        )
        
        y = ( torch.tensor([self.data.isel(time=[idx_d + self.delta_t * self.len_sqce]).values[:,:,:self.out_features] for idx_d in idx_data],\
                         dtype=torch.float).reshape(len(idx), self.nodes, -1), 
             torch.tensor([self.data.isel(time=[idx_d + self.delta_t * (self.len_sqce+1)]).values[:,:,:self.out_features] for idx_d in idx_data],\
                         dtype=torch.float).reshape(len(idx), self.nodes, -1)
        
        )
        
        #X = (torch.tensor(self.data.isel(time=self.idxs[idx][0][0]).values).float().permute(1, 0, 2), 
        #     torch.tensor(self.data.isel(time=self.idxs[idx][0][1]).values[:, self.out_features:]).float())
        
        #y = (torch.Tensor(self.data.isel(time=self.idxs[idx][1][0]).values[:, :self.out_features]).float(), 
        #     torch.Tensor(self.data.isel(time=self.idxs[idx][1][1]).values[:, :self.out_features]).float())
        
        return X, y 

class WeatherBenchDatasetXarrayHealpix(Dataset):
    """ Dataset used for graph models (1D), where data is loaded from an xarray Dataset
    
    Parameters
    ----------
    ds : xarray Dataset
        Dataset containing the input data
    lead_time : int
        Prediction interval (in hours)
    out_features : int
        Number of output features
    years : tuple(str)
        Years used to split the data
    nodes : float
        Number of nodes each sample has
    max_lead_time : int
        Maximum lead time (in case of iterative predictions) in hours
    load : bool
        If true, load dataset to RAM
    mean : np.ndarray of shape 2
        Mean to use for data normalization. If None, mean is computed from data
    std : np.ndarray of shape 2
        std to use for data normalization. If None, mean is computed from data
    """
    
    def __init__(self, ds, out_features, lead_time, years, nodes, nb_timesteps, max_lead_time=None, 
                 load=True, mean=None, std=None):
        
        self.lead_time = lead_time
        self.years = years
        self.nodes = nodes
        self.out_features = out_features
        self.max_lead_time = max_lead_time
        self.nb_timesteps = nb_timesteps
        
        #self.data = ds
        
        self.data = ds.to_array(dim='level', name='Dataset').transpose('time', 'node', 'level')
        self.in_features = self.data.shape[-1]
        
        self.mean = self.data.mean(('time', 'node')).compute() if mean is None else mean
        self.std = self.data.std(('time', 'node')).compute() if std is None else std
        
        if max_lead_time is None:
            self.n_samples = self.data.isel(time=slice(0, -self.nb_timesteps*lead_time)).shape[0]
        else:
            self.n_samples = self.data.isel(time=slice(0, -self.nb_timesteps*lead_time)).shape[0] - max_lead_time
        self.idxs = np.arange(self.n_samples)
        
        # Normalize
        self.data = (self.data - self.mean.to_array(dim='level')) / self.std.to_array(dim='level')
        
        if load: 
            print('Loading data into RAM')
            self.data.load()
        
    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        idxs = self.idxs[idx]
        
        X = (torch.Tensor(self.data.isel(time=idxs).values[:,:, 0].transpose()),
             torch.Tensor(self.data.isel(time=idxs + self.lead_time).values[:,:, 0].transpose()[self.out_features:]))
        
        y = (torch.Tensor(self.data.isel(time=idxs + self.lead_time).values[:,:, 0].transpose()[:self.out_features]), 
             torch.Tensor(self.data.isel(time=idxs + 2*self.lead_time).values[:,:, 0].transpose()[:self.out_features]))

        return X, y

In [6]:
z500 = xr.open_mfdataset(f'{input_dir}geopotential_500/*.nc', combine='by_coords', chunks={'time':chunk_size}).rename({'z':'z500'})
t850 = xr.open_mfdataset(f'{input_dir}temperature_850/*.nc', combine='by_coords', chunks={'time':chunk_size}).rename({'t':'t850'})
rad = xr.open_mfdataset(f'{input_dir}toa_incident_solar_radiation/*.nc', combine='by_coords', chunks={'time':chunk_size})

z500 = z500.isel(time=slice(7, None))
t850 = t850.isel(time=slice(7, None))

constants = xr.open_dataset(f'{input_dir}constants/constants_5.625deg.nc').rename({'orography' :'orog'})
constants = constants.assign(cos_lon=lambda x: np.cos(np.deg2rad(x.lon)))
constants = constants.assign(sin_lon=lambda x: np.sin(np.deg2rad(x.lon)))

temp = xr.DataArray(np.zeros(z500.dims['time']), coords=[('time', z500.time.values)])
constants, _ = xr.broadcast(constants, temp)

orog = constants['orog']
lsm = constants['lsm']
lats = constants['lat2d']
slt = constants['slt']
cos_lon = constants['cos_lon']
sin_lon = constants['sin_lon']

In [7]:
#description = "no_const"
description = "all_const"

model_filename = model_save_path + "spherical_unet_" + description + ".h5"
pred_filename = pred_save_path + "spherical_unet_" + description + ".nc"
rmse_filename = datadir + 'metrics/rmse_' + description + '.nc'

# z500, t850, orog, lats, lsm, slt, rad
#feature_idx = [0, 1]
in_features = 7 #len(feature_idx)
#ds = xr.merge([z500, t850], compat='override')
ds = xr.merge([z500, t850, orog, lats, lsm, slt, rad], compat='override')

ds_train = ds.sel(time=slice(*train_years))
ds_valid = ds.sel(time=slice(*val_years))
ds_test = ds.sel(time=slice(*test_years))


In [8]:
train_mean_ = xr.open_mfdataset(f'{input_dir}mean_train_full_all_const.nc')
train_std_ = xr.open_mfdataset(f'{input_dir}mean_train_full_all_const.nc')

In [17]:
# Train and validation data
training_ds = WeatherBenchDatasetXarrayHealpixTemp(ds=ds_train, out_features=out_features, delta_t=delta_t,
                                                   len_sqce=len_sqce, max_lead_time=max_lead_time,
                                                   years=train_years, nodes=nodes, nb_timesteps=nb_timesteps, 
                                                   mean=train_mean_, std=train_std_, load=False)
validation_ds = WeatherBenchDatasetXarrayHealpixTemp(ds=ds_valid, out_features=out_features, delta_t=delta_t,
                                                     len_sqce=len_sqce, max_lead_time=max_lead_time,
                                                     years=train_years, nodes=nodes, nb_timesteps=nb_timesteps, 
                                                     mean=train_mean_, std=train_std_, load=False)

dl_train = DataLoader(training_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers,\
                      pin_memory=pin_memory)

dl_val = DataLoader(validation_ds, batch_size=batch_size*2, shuffle=False, num_workers=num_workers,\
                    pin_memory=pin_memory)

In [10]:
# Model
spherical_unet = UNetSphericalHealpix(N=nodes, in_channels=in_features*len_sqce, out_channels=out_features, 
                                      kernel_size=3)
spherical_unet, device = init_device(spherical_unet, gpu=gpu)

In [11]:
torch.cuda.empty_cache()

In [15]:
def train_model_2steps_custom(model, device, training_ds, batch_size, epochs, lr, validation_data):    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, eps=1e-7, weight_decay=0, amsgrad=False)
    
    train_losses = []
    val_losses = []
    n_samples = training_ds.n_samples
    
    for epoch in range(epochs):
        
        print('\rEpoch : {}'.format(epoch), end="")
        
        time1 = time.time()
        
        val_loss = 0
        train_loss = 0
        
        model.train()  
        
        random.shuffle(training_ds.idxs)
        idxs = training_ds.idxs
        
        batch_idx = 0
        for i in range(0, n_samples - batch_size, batch_size):
            i_next = min(i + batch_size, n_samples)
        # for batch_idx, (batch, labels) in enumerate(train_generator):
            batch, labels = training_ds[idxs[i:i_next]]
            
            
            # Transfer to GPU
            print(batch[0].shape)
            print(batch[1].shape)
            print(labels[0].shape)
            print(labels[1].shape)
            batch1 = batch[0].permute(2,1,0).to(device)
            constants1 = batch[1].permute(2,1,0).to(device)
            label1 = labels[0].permute(2,1,0).to(device)
            label2 = labels[1].permute(2,1,0).to(device)
            
            batch_size = batch1.shape[0]
            
            #print(batch[0].shape)
            #print(labels[0].shape)
            
            #print(batch1.shape)
            # Model
            output1 = model(batch1)
            #print(output1.shape)
            #print(batch1.shape)
            #print(constants1.shape)
            batch2 = torch.cat((output1, constants1), dim=2)
            output2 = model(batch2)
            
            #print(output1.shape)
            #print(output2.shape)
            #print(label1.shape)
            #print(label2.shape)
            loss = criterion(output1, label1) + criterion(output2, label2)
            
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss = train_loss + loss.item() * batch_size
            
            print('\rBatch idx: {}; Loss: {:.3f}'.format(batch_idx, train_loss/(batch_size*(batch_idx+1))), end="")
            batch_idx += 1
            
        if epoch == 2:
            return output1, output2, label1, label2
        
        train_loss = train_loss / (len(train_generator.dataset))
        train_losses.append(train_loss)
        
        model.eval()
        with torch.set_grad_enabled(False):
            index = 0
            
            for batch, labels in validation_data:
                # Transfer to GPU
                batch1 = batch[0].to(device)
                constants1 = batch[1].to(device)
                label1 = labels[0].to(device)
                label2 = labels[1].to(device)

                batch_size = batch1.shape[0]
                
                output1 = model(batch1)
                batch2 = torch.cat((output1, constants1), dim=2)
                output2 = model(batch2)
                
                val_loss = val_loss + (criterion(output1, label1).item() 
                                       + criterion(output2, label2).item()) * batch_size
                index = index + batch_size
                
        val_loss = val_loss / (len(validation_data.dataset))
        val_losses.append(val_loss)
        
        time2 = time.time()
        
        # Print stuff
        print('Epoch: {e:3d}/{n_e:3d}  - loss: {l:.3f}  - val_loss: {v_l:.5f}  - time: {t:2f}'
              .format(e=epoch+1, n_e=epochs, l=train_loss, v_l=val_loss, t=time2-time1))
        
    return train_losses, val_losses

In [13]:
training_ds.data.isel(time=[0,1,2,3])

<xarray.DataArray (time: 4, node: 3072, level: 7)>
dask.array<getitem, shape=(4, 3072, 7), dtype=float64, chunksize=(4, 3072, 1), chunktype=numpy.ndarray>
Coordinates:
  * level    (level) <U5 'z500' 't850' 'orog' 'lat2d' 'lsm' 'slt' 'tisr'
  * time     (time) datetime64[ns] 1979-01-01T07:00:00 ... 1979-01-01T10:00:00
    lat      (node) float64 dask.array<chunksize=(3072,), meta=np.ndarray>
    lon      (node) float64 dask.array<chunksize=(3072,), meta=np.ndarray>
  * node     (node) int64 0 1 2 3 4 5 6 7 ... 3065 3066 3067 3068 3069 3070 3071

In [18]:
o1, o2, l1, l2 = train_model_2steps_custom(spherical_unet, device, training_ds, batch_size, epochs=7, \
                                           lr=learning_rate, validation_data=dl_val)

Epoch : 0torch.Size([95, 3072, 14])
torch.Size([95, 3072, 5])
torch.Size([95, 3072, 2])
torch.Size([95, 3072, 2])


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/bolon/miniconda3/envs/weather/lib/python3.7/site-packages/torch/nn/parallel/parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "/home/bolon/miniconda3/envs/weather/lib/python3.7/site-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "/mnt/scratch/students/bolon/weather_prediction/modules/healpix_models.py", line 209, in forward
    x_encoded = self.encode(x)
  File "/mnt/scratch/students/bolon/weather_prediction/modules/healpix_models.py", line 136, in encode
    x_enc1 = self.conv11(x)
  File "/home/bolon/miniconda3/envs/weather/lib/python3.7/site-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "/mnt/scratch/students/bolon/weather_prediction/modules/healpix_models.py", line 59, in forward
    x = self.conv(x)
  Fi